In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import optuna

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import warnings
warnings.filterwarnings('ignore')

# Configuration

In [4]:
ROOT = os.path.join("..","..")
INPUT = "input"
LISH_MOA = "lish-moa"
NUM_FOLD = 5
NUM_OPTUNA_TRIAL = 30
N_COMP_GENES = 50
N_COMP_CELLS = 15

# Read data

In [5]:
train = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_features.csv"))
test = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "test_features.csv"))
train_targets_scored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_scored.csv"))
train_targets_nonscored = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "train_targets_nonscored.csv"))
sub = pd.read_csv(os.path.join(ROOT, INPUT, LISH_MOA, "sample_submission.csv"))

In [6]:
train_targets_scored.columns

Index(['sig_id', '5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor',
       'acat_inhibitor', 'acetylcholine_receptor_agonist',
       'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor',
       'adenosine_receptor_agonist', 'adenosine_receptor_antagonist',
       'adenylyl_cyclase_activator',
       ...
       'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist',
       'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor',
       'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b',
       'vitamin_d_receptor_agonist', 'wnt_inhibitor'],
      dtype='object', length=207)

In [7]:
for col in train_targets_scored.columns:
    if col != 'sig_id':
        c = train_targets_scored[col].value_counts()
        if c[1] <= NUM_FOLD:
            print(col)

atp-sensitive_potassium_channel_antagonist
erbb2_inhibitor


## train_features.csv

In [8]:
print(train.shape)

(23814, 876)


In [9]:
train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


## train_targets_socred.csv

In [10]:
print(train_targets_scored.shape)

(23814, 207)


In [11]:
train_targets_scored.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## test_features.csv

In [12]:
print(test.shape)

(3982, 876)


In [13]:
test.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


## train_targets_nonscored.csv (not use)

In [14]:
print(train_targets_nonscored.shape)

(23814, 403)


In [15]:
train_targets_nonscored.head()

,sig_id,abc_transporter_expression_enhancer,abl_inhibitor,ace_inhibitor,acetylcholine_release_enhancer,adenosine_deaminase_inhibitor,adenosine_kinase_inhibitor,adenylyl_cyclase_inhibitor,age_inhibitor,alcohol_dehydrogenase_inhibitor,...,ve-cadherin_antagonist,vesicular_monoamine_transporter_inhibitor,vitamin_k_antagonist,voltage-gated_calcium_channel_ligand,voltage-gated_potassium_channel_activator,voltage-gated_sodium_channel_blocker,wdr5_mll_interaction_inhibitor,wnt_agonist,xanthine_oxidase_inhibitor,xiap_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## submission.csv

In [16]:
sub.head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_0004d9e33,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,id_001897cda,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,id_002429b5b,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,id_00276f245,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,id_0027f1083,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


# Preprocessing

## PCA features + Existing features

In [17]:
def make_pca_features(df_train:pd.DataFrame, df_test:pd.DataFrame, n_components:int, use_cols:list, gene_or_cell:str, concat_flg:bool):
    data = pd.concat([pd.DataFrame(df_train[use_cols]), pd.DataFrame(df_test[use_cols])])
    data_pca = PCA(n_components=n_components, random_state=334).fit_transform(data[use_cols])

    train_pca = data_pca[:df_train.shape[0]]
    test_pca = data_pca[-df_test.shape[0]:]

    train_pca = pd.DataFrame(train_pca, columns=['pca_'+gene_or_cell+str(i) for i in range(n_components)])
    test_pca = pd.DataFrame(test_pca, columns=['pca_'+gene_or_cell+str(i) for i in range(n_components)])

    if concat_flg:
        ret_df_train = pd.concat([df_train, train_pca], axis=1)
        ret_df_test = pd.concat([df_test, test_pca], axis=1)
    else:
        ret_df_train = pd.concat([df_train['sig_id'], train_pca], axis=1)
        ret_df_test = pd.concat([df_test['sig_id'], test_pca], axis=1)
    return ret_df_train, ret_df_test

In [18]:
GENES = [col for col in train.columns if col.startswith('g-')]
CELLS = [col for col in train.columns if col.startswith('c-')]

In [19]:
train, test = make_pca_features(train, test, N_COMP_GENES, GENES, 'G', True)
train, test = make_pca_features(train, test, N_COMP_CELLS, GENES, 'C', True)

## Label Encoding

In [20]:
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, encode_cols):
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in encode_cols:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [21]:
train.select_dtypes(include=['object']).columns

Index(['sig_id', 'cp_type', 'cp_dose'], dtype='object')

In [22]:
# train['cp_type'] = train['cp_type'].astype(str)
# train['cp_dose'] = train['cp_dose'].astype(str)
# test['cp_type'] = test['cp_type'].astype(str)
# test['cp_dose'] = test['cp_dose'].astype(str)

In [23]:
train, test = label_encoding(train, test, ['cp_type', 'cp_dose'])

## Standrization

In [24]:
tr_mean = train.iloc[:, 1:].mean()
tr_std = train.iloc[:, 1:].std()
train.iloc[:, 1:] = (train.iloc[:, 1:] - tr_mean) / tr_std
test.iloc[:, 1:] = (test.iloc[:, 1:] - tr_mean) / tr_std

In [25]:
train.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,pca_C5,pca_C6,pca_C7,pca_C8,pca_C9,pca_C10,pca_C11,pca_C12,pca_C13,pca_C14
0,id_000644bb2,0.291574,-1.237973,-0.980022,0.583920,0.804300,-0.386349,-0.739750,-0.243919,-0.740354,...,-1.037049,-0.898119,0.710287,0.725591,-0.684195,0.370832,-0.116008,0.008490,-0.381942,0.494600
1,id_000779bfc,0.291574,1.235896,-0.980022,-0.124922,0.620885,0.141781,-0.022706,0.931752,0.559218,...,0.994008,0.573591,0.456139,0.080461,0.403450,-0.127276,0.728334,-0.086435,-0.595905,-0.608176
2,id_000a6266a,0.291574,-0.001039,-0.980022,0.272452,0.833843,1.353388,-0.166705,-0.086859,1.168263,...,-0.363874,0.630584,-0.177126,-0.464212,-1.402568,0.076530,-0.393608,-0.207932,0.172624,0.920825
3,id_0015fd391,0.291574,-0.001039,-0.980022,-0.546984,-0.188852,-0.403438,0.470340,3.880135,-0.568655,...,-2.174904,1.328472,-0.534099,-0.043367,1.410677,-0.857507,0.151173,-0.943993,0.997108,0.075326
4,id_001626bd3,0.291574,1.235896,1.020342,-0.411775,-0.375714,0.789535,0.642022,1.318346,-0.581288,...,-0.910468,-0.390191,0.147714,-1.810480,1.653665,0.161974,0.640640,-0.601830,1.307257,0.276329


In [26]:
test.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,pca_C5,pca_C6,pca_C7,pca_C8,pca_C9,pca_C10,pca_C11,pca_C12,pca_C13,pca_C14
0,id_0004d9e33,0.291574,-1.237973,-0.980022,-0.569949,0.278550,-0.642786,0.377709,1.446241,-0.021676,...,-0.319894,0.046414,-0.038783,-0.603362,0.436967,-0.381984,-0.410147,-0.292527,-0.172250,0.524562
1,id_001897cda,0.291574,1.235896,-0.980022,-0.309507,0.403371,1.019325,-0.562279,-0.409408,-0.163699,...,-1.259322,-1.953961,0.216240,-0.170414,0.482602,1.072487,0.324963,-0.759370,-0.135028,0.649987
2,id_002429b5b,-3.429514,-1.237973,-0.980022,-0.045333,-0.055044,-0.524608,0.051608,-1.448851,0.325877,...,1.164812,0.485769,-0.908985,0.657602,0.411679,1.017643,-0.302356,0.191476,0.151532,-0.080728
3,id_00276f245,0.291574,-1.237973,1.020342,0.168246,0.358440,0.222303,0.360447,-0.622858,-0.901454,...,0.371059,-0.051105,-0.373819,0.890438,-0.624379,0.645688,-0.310583,-0.414523,0.316770,-0.477419
4,id_0027f1083,0.291574,-0.001039,-0.980022,-0.463806,-1.443093,1.700003,0.130239,-0.623731,0.103643,...,0.901600,0.759871,-0.466667,0.288192,0.531159,0.382763,-0.268626,0.476938,-0.066478,0.288976


# Cross Validation by using Optuna

In [27]:
def logistic_elasticnet_cv(data, targets, target_col, cv):
    def objective(trial):
        param = {
            'penalty': 'l1',
            'C': trial.suggest_loguniform('C', 2**(-10), 2**5),
            'solver': 'liblinear',
            'n_jobs': 8
        }
        
        df_x = data.drop(["sig_id"], axis=1)
        df_y = targets[target_col]
        score_logloss = 0.
        
        # cross validation
        for fold_id, (train_index, valid_index) in enumerate(tqdm(cv.split(df_x, df_y))):
            X_train = df_x.loc[train_index, :]
            y_train = df_y[train_index]
            X_valid = df_x.loc[valid_index, :]
            y_valid = df_y.loc[valid_index]
            
            model = LogisticRegression(**param)
            model.fit(X_train, y_train)
            y_pred = model.predict_proba(X_valid)
            score_logloss += log_loss(y_valid, y_pred, labels=[0, 1])

        score_logloss /= NUM_FOLD
        return score_logloss
    return objective
        

In [28]:
cv = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=0)

In [29]:
skip_cols = ["atp-sensitive_potassium_channel_antagonist", "erbb2_inhibitor"]

In [30]:
for target_col in tqdm(train_targets_scored.columns[150:]):
    if (target_col != "sig_id") and not (target_col in skip_cols):
        print('##################### CV START: {0} #####################'.format(target_col))
        
        # optimize by optuna
        study = optuna.create_study()
        study.optimize(logistic_elasticnet_cv(train, train_targets_scored, target_col, cv), n_trials=NUM_OPTUNA_TRIAL)
        
        # save optuna log
        df_trial = study.trials_dataframe()
        df_trial.to_pickle(os.path.join('report', 'LR_PCA_OptunaLog_'+target_col+'.pkl'))

        # save best params found by CV
        bestparam_filename = 'LR_PCA_BestParamsSelectedByCV_'+target_col+'.pkl'
        with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
            pickle.dump(study.best_params, f)


  0%|                                                                                           | 0/57 [00:00<?, ?it/s]

##################### CV START: pdgfr_inhibitor #####################



0it [00:00, ?it/s]
1it [00:03,  3.79s/it]
2it [00:07,  3.73s/it]
3it [00:10,  3.63s/it]
4it [00:13,  3.50s/it]
5it [00:17,  3.57s/it]
[I 2020-10-30 20:18:41,407] Finished trial#0 with value: 0.024781448876424927 with parameters: {'C': 0.29634856958875117}. Best is trial#0 with value: 0.024781448876424927.

0it [00:00, ?it/s]
1it [00:06,  6.04s/it]
2it [00:11,  5.89s/it]
3it [00:17,  5.83s/it]
4it [00:22,  5.70s/it]
5it [00:28,  5.76s/it]
[I 2020-10-30 20:19:10,330] Finished trial#1 with value: 0.02942960180190603 with parameters: {'C': 0.6182828035143116}. Best is trial#0 with value: 0.024781448876424927.

0it [00:00, ?it/s]
1it [00:14, 14.25s/it]
2it [00:28, 14.16s/it]
3it [00:43, 14.54s/it]
4it [00:59, 14.87s/it]
5it [01:15, 15.16s/it]
[I 2020-10-30 20:20:26,264] Finished trial#2 with value: 0.05825881761358742 with parameters: {'C': 11.367381161629345}. Best is trial#0 with value: 0.024781448876424927.

0it [00:00, ?it/s]
1it [00:00,  1.57it/s]
2it [00:01,  1.55it/s]
3it [00:01,  1

##################### CV START: pdk_inhibitor #####################



0it [00:00, ?it/s]
1it [00:00,  1.56it/s]
2it [00:01,  1.54it/s]
3it [00:01,  1.52it/s]
4it [00:02,  1.53it/s]
5it [00:03,  1.52it/s]
[I 2020-10-30 20:30:24,392] Finished trial#0 with value: 0.00731231718669874 with parameters: {'C': 0.02459015765036835}. Best is trial#0 with value: 0.00731231718669874.

0it [00:00, ?it/s]
1it [00:00,  1.62it/s]
2it [00:01,  1.60it/s]
3it [00:01,  1.58it/s]
4it [00:02,  1.59it/s]
5it [00:03,  1.58it/s]
[I 2020-10-30 20:30:27,671] Finished trial#1 with value: 0.014582902146124201 with parameters: {'C': 0.0050603020184390056}. Best is trial#0 with value: 0.00731231718669874.

0it [00:00, ?it/s]
1it [00:01,  1.49s/it]
2it [00:05,  2.12s/it]
3it [00:06,  1.93s/it]
4it [00:09,  2.12s/it]
5it [00:14,  2.99s/it]
[I 2020-10-30 20:30:42,759] Finished trial#2 with value: 0.006528884428050723 with parameters: {'C': 0.4555579165093862}. Best is trial#2 with value: 0.006528884428050723.

0it [00:00, ?it/s]
1it [00:01,  1.26s/it]
2it [00:03,  1.70s/it]
3it [00:05, 

...


0it [00:00, ?it/s]
1it [00:01,  1.71s/it]
2it [00:03,  1.73s/it]
3it [00:05,  1.74s/it]
4it [00:07,  1.75s/it]
5it [00:08,  1.76s/it]
[I 2020-10-31 03:54:22,327] Finished trial#0 with value: 0.009387336357522872 with parameters: {'C': 0.44314873317583703}. Best is trial#0 with value: 0.009387336357522872.

0it [00:00, ?it/s]
1it [00:04,  4.43s/it]
2it [00:08,  4.33s/it]
3it [00:12,  4.36s/it]
4it [00:17,  4.28s/it]
5it [00:21,  4.31s/it]
[I 2020-10-31 03:54:44,016] Finished trial#1 with value: 0.014345666901904955 with parameters: {'C': 6.543668258871826}. Best is trial#0 with value: 0.009387336357522872.

0it [00:00, ?it/s]
1it [00:00,  1.70it/s]
2it [00:01,  1.67it/s]
3it [00:01,  1.65it/s]
4it [00:02,  1.66it/s]
5it [00:03,  1.65it/s]
[I 2020-10-31 03:54:47,153] Finished trial#2 with value: 0.028629110896528566 with parameters: {'C': 0.002261973401733781}. Best is trial#0 with value: 0.009387336357522872.

0it [00:00, ?it/s]
1it [00:01,  1.03s/it]
2it [00:02,  1.02s/it]
3it [00:02,

##################### CV START: vitamin_d_receptor_agonist #####################



0it [00:00, ?it/s]
1it [00:00,  1.65it/s]
2it [00:01,  1.64it/s]
3it [00:01,  1.63it/s]
4it [00:02,  1.63it/s]
5it [00:03,  1.64it/s]
[I 2020-10-31 03:58:09,738] Finished trial#0 with value: 0.028940351759974125 with parameters: {'C': 0.0025151653453235738}. Best is trial#0 with value: 0.028940351759974125.

0it [00:00, ?it/s]
1it [00:02,  2.10s/it]
2it [00:04,  2.11s/it]
3it [00:06,  2.16s/it]
4it [00:08,  2.09s/it]
5it [00:10,  2.10s/it]
[I 2020-10-31 03:58:20,353] Finished trial#1 with value: 0.004172483799436422 with parameters: {'C': 1.101426581371618}. Best is trial#1 with value: 0.004172483799436422.

0it [00:00, ?it/s]
1it [00:00,  1.36it/s]
2it [00:01,  1.37it/s]
3it [00:02,  1.37it/s]
4it [00:02,  1.33it/s]
5it [00:03,  1.35it/s]
[I 2020-10-31 03:58:24,182] Finished trial#2 with value: 0.012068089034094925 with parameters: {'C': 0.0319311448805895}. Best is trial#1 with value: 0.004172483799436422.

0it [00:00, ?it/s]
1it [00:00,  1.07it/s]
2it [00:01,  1.05it/s]
3it [00:02,

##################### CV START: wnt_inhibitor #####################



0it [00:00, ?it/s]
1it [00:04,  4.98s/it]
2it [00:09,  4.80s/it]
3it [00:14,  4.89s/it]
4it [00:18,  4.70s/it]
5it [00:22,  4.57s/it]
[I 2020-10-31 04:03:21,148] Finished trial#0 with value: 0.0174053236766161 with parameters: {'C': 5.9161994617507965}. Best is trial#0 with value: 0.0174053236766161.

0it [00:00, ?it/s]
1it [00:00,  1.22it/s]
2it [00:01,  1.22it/s]
3it [00:02,  1.23it/s]
4it [00:03,  1.19it/s]
5it [00:04,  1.22it/s]
[I 2020-10-31 04:03:25,366] Finished trial#1 with value: 0.00978183032984135 with parameters: {'C': 0.09640100489315091}. Best is trial#1 with value: 0.00978183032984135.

0it [00:00, ?it/s]
1it [00:00,  1.64it/s]
2it [00:01,  1.64it/s]
3it [00:01,  1.63it/s]
4it [00:02,  1.62it/s]
5it [00:03,  1.63it/s]
[I 2020-10-31 04:03:28,543] Finished trial#2 with value: 0.04872593478410107 with parameters: {'C': 0.001232368443990199}. Best is trial#1 with value: 0.00978183032984135.

0it [00:00, ?it/s]
1it [00:00,  1.66it/s]
2it [00:01,  1.65it/s]
3it [00:01,  1.64i

# Train & Predict by best model

In [35]:
submission = sub.copy()

In [36]:
def logistic_elasticnet_bestparam(data_train, targets_train, data_test, target_col, best_params, submission):
    print('##################### TraingByBestParam START: {0} #####################'.format(target_col))
    
    # prepare data set
    X_train = data_train.drop(["sig_id"], axis=1)
    y_train = targets_train[target_col]
    X_test = data_test.drop(["sig_id"], axis=1)
    
    # train XGBoost by using best_params
    model = LogisticRegression(**best_params)
    model.fit(X_train, y_train)
    
    # predict for test
    submission[target_col] = model.predict_proba(X_test)[:, 1]
    
    # save model for test
    modelfile = 'LR_PCA_BestModel4test_'+target_col+'.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    

In [37]:
# for target_col in tqdm(train_targets_scored.columns):
for target_col in tqdm(train_targets_scored.columns[150:]):
    if target_col != "sig_id" and not (target_col in skip_cols):
        # read best param selected by CV
        bestparam_filename = 'LR_PCA_BestParamsSelectedByCV_'+target_col+'.pkl'
        with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'rb') as f:
            best_params = pickle.load(f)
            
        best_params['penalty'] = 'l1'
        best_params['n_jobs'] = 8
        best_params['solver'] = 'liblinear'
        
        logistic_elasticnet_bestparam(train, train_targets_scored, test, target_col, best_params, submission)
    elif target_col in skip_cols:
        submission[target_col] = 1e-05

  0%|                                                                                           | 0/57 [00:00<?, ?it/s]

##################### TraingByBestParam START: pdgfr_inhibitor #####################


  2%|█▍                                                                                 | 1/57 [00:03<03:10,  3.40s/it]

##################### TraingByBestParam START: pdk_inhibitor #####################


  4%|██▉                                                                                | 2/57 [00:04<02:30,  2.73s/it]

...

 96%|███████████████████████████████████████████████████████████████████████████████   | 55/57 [01:39<00:03,  1.82s/it]

##################### TraingByBestParam START: vitamin_d_receptor_agonist #####################


 98%|████████████████████████████████████████████████████████████████████████████████▌ | 56/57 [01:42<00:02,  2.11s/it]

##################### TraingByBestParam START: wnt_inhibitor #####################


100%|██████████████████████████████████████████████████████████████████████████████████| 57/57 [01:43<00:00,  1.82s/it]


In [34]:
submission.to_csv('submission.csv', index=False)